In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report


In [55]:
binned = pd.read_csv('/Users/flatironschool/Absenteeism_Project/data/processed/binned.csv')


/anaconda3/envs/absenteeism/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [56]:
binned.head()

,Unnamed: 0,index,Unnamed: 0.1,Unnamed: 0.1.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,...,total_suspension_days,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin
0,0,0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,...,78.0,0.060232,0.0,0.000000,2811.937359,0.0,0.003475,0.378788,92,90%+
1,1,1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,95,...,10.0,0.018553,2.0,0.003711,4825.189777,0.0,0.002783,0.100000,95,90%+
2,2,2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,...,18.0,0.030303,5.0,0.008418,5317.932795,0.0,0.001684,0.105263,85,80-89%
3,3,3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,...,10.0,0.021786,0.0,0.000000,5909.375686,0.0,0.002179,0.068966,80,80-89%
4,4,4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,...,8.0,0.022039,0.0,0.000000,3962.305785,0.0,0.002755,0.101010,85,80-89%


## Create Four Bins

In [57]:
#create the binned categories
binned['four_rate_bins'] = pd.cut(binned['grad_slice'].astype(int), 
    [0, 59, 79, 89, 100], labels = ['0-59%', '60-79%', '80-89%', '90%+'])

In [7]:
binned.tail()

,Unnamed: 0,index,Unnamed: 0.1,Unnamed: 0.1.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,...,suspensed_day_rate,harassed,harassed_rate,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin,four_rate_bins
16568,16568,21854,21854,21854,WYOMING,Sheridan County School District #1,560569000311,Big Horn High School,37,90,...,0.013245,4.0,0.026490,1738.913907,0.0,0.006623,0.846154,90,90%+,90%+
16569,16569,21858,21858,21858,WYOMING,Sheridan County School District #2,560569500360,Sheridan High School,236,89,...,0.046796,0.0,0.000000,1694.559176,0.0,0.005086,0.198779,89,80-89%,80-89%
16570,16570,21861,21861,21861,WYOMING,Sweetwater County School District #2,560576200324,Green River High School,176,85-89,...,0.141570,1.0,0.001287,1840.877490,0.0,0.003861,0.200000,85,80-89%,80-89%
16571,16571,21863,21863,21863,WYOMING,Teton County School District #1,560583000335,Jackson Hole High School,127,95,...,0.040625,0.0,0.000000,4153.518984,0.0,0.004687,0.118939,95,90%+,90%+
16572,16572,21866,21866,21866,WYOMING,Washakie County School District #1,560624000343,Worland High School,105,75-79,...,0.025000,0.0,0.000000,3015.755325,0.0,0.004275,0.331544,75,70-79%,60-79%


In [58]:
binned['four_rate_bins'].value_counts()

90%+      8712
80-89%    4257
60-79%    1993
0-59%     1611
Name: four_rate_bins, dtype: int64

## Impute Mean for Missing Values

In [59]:
#impute mean for feature numerical vars
#did not include total enrollment or all rate
binned_feat = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [60]:
for i in range(0, len(binned_feat.columns)):
    print('im here:', binned_feat.columns[i])
    binned.fillna(binned.mean(), inplace=True)

im here: ap_ib_de_rate
im here: sat_act_rate
im here: pass_algebra_rate
im here: geometry_rate
im here: algebra2_rate
im here: calc_rate
im here: chronic_absent_rate
im here: activities_funds_rate
im here: sports_rate
im here: suspensed_day_rate
im here: harassed_rate
im here: non_cert_rate
im here: counselor_rate
im here: absent_teacher_rate


In [61]:
#No missing values
binned.isna().sum()

Unnamed: 0                 0
index                      0
Unnamed: 0.1               0
Unnamed: 0.1.1             0
STNAM                      0
LEANM                      0
NCESSCH                    0
SCHNAM                     0
ALL_COHORT_1516            0
ALL_RATE_1516              0
LEA_STATE                  0
LEA_STATE_NAME             0
LEAID_y                    0
LEA_NAME                   0
SCHID                      0
SCH_NAME                   0
COMBOKEY                   0
JJ                         0
SCH_STATUS_SPED            0
SCH_STATUS_MAGNET          0
SCH_STATUS_CHARTER         0
SCH_STATUS_ALT             0
SCH_MAGNETDETAIL         638
SCH_ALTFOCUS             498
TOT_ENR_M                  0
TOT_ENR_F                  0
TOT_GTENR_M                0
TOT_GTENR_F                0
TOT_DUAL_M                 0
TOT_DUAL_F                 0
                        ... 
districtID                 0
IDSCH                      0
total_enrollment           0
total_ap_ib_de

## Model 1 - Random Forest Classifier

In [12]:
y = binned['four_rate_bins']
X = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [13]:
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.33, random_state=42)


In [45]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5)

In [46]:
rf.fit(X_train_rf, y_train_rf)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [47]:
rf.score(X_train_rf, y_train_rf)

0.6273980005403945

In [48]:
rf.score(X_test_rf, y_test_rf)

0.6113345521023766

In [49]:
confusion_matrix(y_test_rf, rf.predict(X_test_rf))

array([[ 334,   28,   27,  188],
       [  88,   33,  177,  337],
       [  47,   13,  238, 1080],
       [  27,    3,  111, 2739]])

## Model 2 - Random Forest with Class Weights

In [50]:
y = binned['four_rate_bins']
X = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [78]:
#with only four bins, dummies variables to use class weights becomes an option
pd.get_dummies(data = binned, columns= ['four_rate_bins'])

,Unnamed: 0,index,Unnamed: 0.1,Unnamed: 0.1.1,STNAM,LEANM,NCESSCH,SCHNAM,ALL_COHORT_1516,ALL_RATE_1516,...,activities_funds_rate,non_cert_rate,counselor_rate,absent_teacher_rate,grad_slice,grad_rate_bin,four_rate_bins_0-59%,four_rate_bins_60-79%,four_rate_bins_80-89%,four_rate_bins_90%+
0,0,0,0,0,ALABAMA,Albertville City,10000500871,Albertville High Sch,296,92,...,2811.937359,0.000000,0.003475,0.378788,92,90%+,0,0,0,1
1,1,1,1,1,ALABAMA,Marshall County,10000600872,Asbury Sch,67,95,...,4825.189777,0.000000,0.002783,0.100000,95,90%+,0,0,0,1
2,2,2,2,2,ALABAMA,Marshall County,10000600878,Douglas High Sch,153,85-89,...,5317.932795,0.000000,0.001684,0.105263,85,80-89%,0,0,1,0
3,3,3,3,3,ALABAMA,Marshall County,10000600883,Kate D Smith DAR High Sch,120,80-84,...,5909.375686,0.000000,0.002179,0.068966,80,80-89%,0,0,1,0
4,4,4,4,4,ALABAMA,Marshall County,10000601585,Brindlee Mt High Sch,94,85-89,...,3962.305785,0.000000,0.002755,0.101010,85,80-89%,0,0,1,0
5,5,5,5,5,ALABAMA,Hoover City,10000700251,Hoover High Sch,714,92,...,434.833734,0.019066,0.001685,0.333651,92,90%+,0,0,0,1
6,6,7,7,7,ALABAMA,Hoover City,10000701456,Spain Park High Sch,412,94,...,511.925343,0.016920,0.001789,0.346870,94,90%+,0,0,0,1
7,7,8,8,8,ALABAMA,Madison City,10000800831,Bob Jones High Sch,451,97,...,72676.817158,0.000000,0.002278,0.019608,97,90%+,0,0,0,1
8,8,9,9,9,ALABAMA,Madison City,10000802198,James Clemens High School,404,96,...,3478.029292,0.000000,0.002635,0.232804,96,90%+,0,0,0,1
9,9,10,10,10,ALABAMA,Leeds City,10001102096,Leeds High Sch,123,90-94,...,3966.648380,0.000000,0.002132,0.343750,90,90%+,0,0,0,1


In [83]:
X_train_weight, X_test_weight, y_train_weight, y_test_weight = train_test_split(X, y, test_size=0.33, random_state=42)


In [181]:
weight={'0-59%': 15, '60-79%': 12, '80-89%':5, '90%+':4}

In [182]:
rf_weight = RandomForestClassifier(n_estimators=400, max_depth=5, 
    class_weight=weight)

In [183]:
rf_weight.fit(X_train_weight, y_train_weight)

RandomForestClassifier(bootstrap=True,
            class_weight={'0-59%': 15, '60-79%': 12, '80-89%': 5, '90%+': 4},
            criterion='gini', max_depth=5, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=None, oob_score=False,
            random_state=None, verbose=0, warm_start=False)

In [184]:
#best score by trying variables, using info to inform grid search
print('RF class weight train score: ', rf_weight.score(X_train_weight, y_train_weight))
print('RF class weight test score: ', rf_weight.score(X_test_weight, y_test_weight))

RF class weight train score:  0.6123570206250563
RF class weight test score:  0.6058500914076782


In [185]:
rf_weight.predict(X_test_weight)
confusion_matrix(y_test_weight, rf_weight.predict(X_test_weight))

array([[ 461,   57,    0,   59],
       [ 151,  240,    4,  240],
       [  88,  318,   12,  960],
       [  90,  180,    9, 2601]])

## Model 3 - Balanced Random Forest

In [63]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [64]:
y = binned['four_rate_bins']
X = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [66]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal = train_test_split(X, y, test_size=0.33, random_state=42)


In [67]:
balance = BalancedRandomForestClassifier(max_depth=2)

In [69]:
balance.fit(X_train_bal, y_train_bal)

BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                criterion='gini', max_depth=2, max_features='auto',
                max_leaf_nodes=None, min_impurity_decrease=0.0,
                min_samples_leaf=2, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
                oob_score=False, random_state=None, replacement=False,
                sampling_strategy='auto', verbose=0, warm_start=False)

In [70]:
balance.score(X_train_bal, y_train_bal)

0.5772313789066018

In [71]:
balance.score(X_test_bal, y_test_bal)

0.5861060329067642

In [72]:
confusion_matrix(y_test_bal, balance.predict(X_test_bal))

array([[ 487,   36,    7,   47],
       [ 190,  182,   86,  177],
       [ 167,  252,  253,  706],
       [ 214,  127,  255, 2284]])

## Model 4 - Grid Search

In [ ]:
y = binned['four_rate_bins']
X = binned[['ap_ib_de_rate', 'sat_act_rate', 'pass_algebra_rate',
      'geometry_rate', 'algebra2_rate', 'calc_rate', 'chronic_absent_rate', 'activities_funds_rate',
      'sports_rate', 'suspensed_day_rate', 'harassed_rate',
      'non_cert_rate','counselor_rate','absent_teacher_rate']]

In [175]:
#use X, y from above
X_train_tune, X_test_tune, y_train_tune, y_test_tune = train_test_split(X, y, test_size=0.2, random_state=102)

In [180]:
# Create the random grid
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 4500, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Class weights
weights={'0-59%': [{0: w} for w in [20, 18, 16, 14, 10]], '60-79%': [{0: w} for w in [17, 15, 13, 11, 9]], '80-89%': [{0: w} for w in [4, 5, 6, 7, 8]], '90%+':[{0: w} for w in [2, 3, 4, 5, 6]]}

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
            'class_weight': weights}


In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_tune, y_train_tune)

In [ ]:
rf_random.best_params_

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = base_model.predict(X_test_tune)
    errors = (predictions - y_test_tune)
    rmse = np.sqrt(np.mean(errors**2)) ### RMSE
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('RMSE = {:0.4f}'.format(rmse))
    
    return rmse

In [ ]:
base_model = RandomForestRegressor(n_estimators = 10)
base_model.fit(X_train_tune, y_train_tune)
base_accuracy = evaluate(base_model, X_test_tune, y_test_tune)